In [17]:
from rk4step import rk4step
import numpy as np
import casadi as ca

In [19]:
Ts = .5
N = 19
x0 = np.array([0])
t_grid = np.linspace(0, Ts*N, N+1)
t_span = [0, Ts*N]
t0 = 0

In [21]:
k = ca.MX.sym('k',1)
x = ca.MX.sym('x',2)
u = ca.MX.sym('u',1)
w = ca.MX.sym('w',1)

k = ca.MX.sym('k',1)
x = ca.MX.sym('x',2)
u = ca.MX.sym('u',1)
w = ca.MX.sym('w',1)

In [16]:
P_e = 10
P_i = 5
C = 50
M = 5

def V(w):
    return w * C 

def O_i(k, O_i_prev, u):
    return O_i_prev + u * M * k

def O_e(k, w, O_i_prev, u):
    return ca.max(V(w) - O_i(k, O_i_prev, u), 0)

def L_e(k, w, x, u):
    return P_e * O_e(k, w, x[0], u)

def L_i(k, x, u):
    P_i * O_i(k, x[0], u)

#L_e = ca.Function('L_e', [k, w, x, u], [L_e])


In [20]:
X_rk4 = np.zeros((x0.size, N+1))
X_rk4[:,0] = x0
for k in range(1, len(t_grid)):
    X_rk4[:,k] = rk4step(L_e, Ts,  X_rk4[:,k-1], t0).full().flatten()

TypeError: L_e() missing 2 required positional arguments: 'x' and 'u'

In [13]:
X_rk4[:,-1]

array([-0.38768787,  0.00873917])

# berechne step_cost_to_go

In [ ]:
M = Tiefe x O x V # matrix
for i in range(Tiefe, 0):
    if (i == Tiefe): #erster step
        berechne M[Tiefe-1, :, :]
    else:
        berechne M[i,:,:] in abhängigkeit von M[i+1,:,:]

step_cost_to_go = M[0;.,:]

# berechne optimalen schritt

In [ ]:
x0 = ...
for u in U:
    array += kosten_für_step(x0, u) + Erwartungswert(step_cost_to_go[f(x0,u)])
step_u = min(array)

In [139]:
A = np.ones(2, dtype=int)

In [141]:
A[0:]

array([1, 1])

In [40]:
from rk4step import rk4step
import numpy as np
import casadi as ca
import random

"""
Ts = .5
N = 19
x0 = np.array([1, 0])
t_grid = np.linspace(0, Ts*N, N+1)
t_span = [0, Ts*N]
t0 = 0


X_rk4 = np.zeros((x0.size, N+1))
X_rk4[:,0] = x0
for k in range(1, len(t_grid)):
    X_rk4[:,k] = rk4step(f, Ts,  X_rk4[:,k-1], t0).full().flatten()
"""


P_e = 20
P_i = 10
C = 50
M = 5



def L(x0, u, v):
    x1 = f(x0, u, v)
    return L_i(x0[0], x1[0]) + L_e(x0[0], x1[0], v)

def f(x0, u, v):
    o1 = x0[0] + u
    v1 = v 
    return (o1, v1)

def L_e(o0, o1, v):
    if(o0 >= v and o1 >= v):
        return 0
    elif(o0 < v and o1 < v):
        return 0.5*(v - o0 + v - o1) * P_e
    else:
        t = (v - o0)/(o1-o0)
        if(o0 < v and o1 >= v):
            return 0.5 * (o1 - v) * (1-t) * P_e
        elif(o0 >= v and o1 < v):
            return 0.5 * (o0 - v) * t * P_e 

def L_i(o0, o1):
    return 0.5 * (o0 + o1) * P_i


"""
def V(v):
    return v * C

def O_i(k, O_i_prev, u):
    return O_i_prev + u * k

def L_i(k, O_i_prev, u):
    return P_i * O_i(k, O_i_prev, u)

def O_e(k, O_i_prev, u, v):
    return np.max([V(v) - O_i(k, O_i_prev, u), 0])

def L_e(k, O_i_prev, u, v):
    return P_e * O_e(k, O_i_prev, u, v)

# integrator wrong
def L(x0, u, v):
    def f(k, v):
        k = ca.SX.sym('k')
        return L_e(k, x0[0], u, v) + L_i(k, x0[0], u)

    x = ca.SX.sym('x')
    ode = {'x':x, 'p':v, 't':k,'ode':f}

    F = ca.integrator('F', 'idas', ode,{'t0':0,'tf':1})
    r = F(x0=0)
    return(r['xf'])
"""


depth = 10
dim_O = 5
dim_V = 5

U = [-2, -1,0,1,2]
O_range = list(range(dim_O))
V_range = list(range(dim_V))
V_realisation = [4,4,4,4,4,1,0,4,2,3,3] #V_realisation = C = np.array([random.randint(min(V_range),max(V_range)) for j in range(depth+1)])

def calculate_cost_to_go_matrix_sequence(V_realisation):
    M = np.zeros((depth,dim_O,dim_V), dtype=float)
    for i in range(depth):
        if(i != 0):
            M[i,:,:] = calculate_cost_to_go_matrix(V_realisation[i], M[i-1,:,:])
        else:
            M[0,:,:] = calculate_cost_to_go_matrix_final_step()

    return M[-1,:,:]

# calculate matrix with includes indexes for actions U that are optimal for given x0 in matrix
def calculate_optimal_step_matrix(V_realisation):
    M_cost_to_go = calculate_cost_to_go_matrix_sequence(V_realisation[1:])
    optimal_steps = np.zeros((dim_O,dim_V), dtype=int)
    for o_index in range(dim_O):
        for v_index in range(dim_V):
            step_cost_to_go_array = np.zeros(len(U), dtype=float)
            for u_index in range(len(U)):
                x0 = (O_range[o_index], V_range[v_index])
                step_cost_to_go_array[u_index] = calculate_path_cost(x0, U[u_index], V_realisation[0], M_cost_to_go)
            min_index_u = 0
            for i in range(1, len(step_cost_to_go_array)):
                if (step_cost_to_go_array[i] < step_cost_to_go_array[min_index_u]):
                    min_index_u = i
            optimal_steps[o_index, v_index] = U[min_index_u]
    return optimal_steps

    # TODO change end score
def calculate_cost_to_go_matrix_final_step():
    M = np.ndarray((dim_O, dim_V), dtype=float)
    for o_index in range(dim_O):
        for v_index in range(dim_V):
            M[o_index,v_index] = 0 # end costs are zeros at the moment
    return M

# TODO optimieren mit list comprehension?
def calculate_cost_to_go_matrix(v_new, M_N_plus_1):
    M = np.ndarray((dim_O, dim_V), dtype=float)
    for o_index in range(dim_O):
        for v_index in range(dim_V):
            M[o_index,v_index] = cost_to_go((O_range[o_index], V_range[v_index]), v_new, M_N_plus_1)
    return M
    # [[cost_to_go((o,v), U, M_N_plus_1) for o in range(dim_O)] for v in range(dim_V)]


def cost_to_go(x0, v_new, M_N_plus_1):
    step_cost_to_go_array = np.zeros(len(U), dtype=float)
    for u_index in range(0,len(U)):
        step_cost_to_go_array[u_index] = calculate_path_cost(x0, U[u_index], v_new, M_N_plus_1)
    return np.min(step_cost_to_go_array)

#TODO ERWARTUNGSWERT
def calculate_path_cost(x0, u, v_new, M_N_plus_1):
    if(x0[0] + u in O_range):
        stage_cost = L(x0, u, v_new)
        prev_cost_to_go = M_N_plus_1[f(x0, u, v_new)]
        return stage_cost + prev_cost_to_go
    else:
        return np.inf

opt_step = calculate_optimal_step_matrix(V_realisation)
print(opt_step)

[[ 0  0  0  0  0]
 [-1 -1 -1 -1 -1]
 [-2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2]]


In [21]:
M = np.zeros((depth,dim_O,dim_V), dtype=float)
for i in range(depth):
    if(i != 0):
        M[i,:,:] = calculate_cost_to_go_matrix(V_realisation[i], M[i-1,:,:])
    else:
        M[0,:,:] = calculate_cost_to_go_matrix_final_step()

In [27]:
M[4,:,:]

array([[7., 7., 7., 7., 7.],
       [4., 4., 4., 4., 4.],
       [1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [41]:
x0 = (3,4)
u = 0
v_new = 4
M_N_plus_1 = calculate_cost_to_go_matrix_final_step()

if(x0[0] + u in O_range):
    print(L(x0, u, v_new))
    print(M_N_plus_1[f(x0, u, v_new)])
else:
    print("out of range")

301.0
0.0
